In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import spacy
from tqdm import tqdm
#nlp = spacy.load('en_core_web_lg', parse=True, tag=True, entity=True)
#nlp = spacy.load('en_core_web_sm',parse=True, tag=True, entity=True)
nlp = spacy.load('en')
import nltk

In [2]:
competitors = ['Chevy','chevy','Ford','ford','Nissan','nissan','Honda','honda','Chevrolet','chevrolet','Volkswagen','volkswagen','benz','Benz','Mercedes','mercedes','subaru','Subaru','VW']

In [3]:
toy_rev = pd.read_csv('csvfile.csv')
toy_rev.head()

toy_rev = toy_rev['Tweets'].to_frame()

C:\Users\sm185567\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 
  

In [5]:
toy_rev = toy_rev.dropna().reset_index()

In [6]:
aspect_terms = []
comp_terms = []
easpect_terms = []
ecomp_terms = []
enemy = []
for x in tqdm(range(len(toy_rev['Tweets']))):
    amod_pairs = []
    advmod_pairs = []
    compound_pairs = []
    xcomp_pairs = []
    neg_pairs = []
    eamod_pairs = []
    eadvmod_pairs = []
    ecompound_pairs = []
    eneg_pairs = []
    excomp_pairs = []
    enemlist = []
    if len(str(toy_rev['Tweets'][x])) != 0:
        lines = nltk.tokenize.sent_tokenize(toy_rev['Tweets'][x])       
        for i in range(len(lines)):
                lines[i] = re.sub('[@#:!;?\.]', ' ',lines[ i])
                lines[i] = re.sub('https.*', ' ',lines[ i])
                lines[i] = re.sub('co.*', ' ',lines[ i])
                lines[i] = lines[i].encode('ascii', 'ignore').decode('ascii')
                #word_tokens = word_tokenize(lines[i])
                #lines[i] = ' '.join(w for w in word_tokens if not w in stop_words)       
        while("" in lines) : 
                lines.remove("")       
        for line in lines:
            enem_list = []
            for eny in competitors:
                enem = re.search(eny,line)
                if enem is not None:
                    enem_list.append(enem.group())
            if len(enem_list)==0:
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                compound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                amod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        amod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    amod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                advmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                neg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                advmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j,h in zip(token.rights,token.lefts):
                            if j.dep_ is 'xcomp' and h.dep_ is not 'neg':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        xcomp_pairs.append(token.text+' '+k.text+' '+j.text)
                            elif j.dep_ is 'xcomp' and h.dep_ is 'neg':
                                if k.dep_ is 'aux':
                                        neg_pairs.append(h.text +' '+token.text+' '+k.text+' '+j.text)
            
            else:
                enemlist.append(enem_list)
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                ecompound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                eamod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        eamod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    eamod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                eadvmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                eneg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                eadvmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j in token.rights:
                            if j.dep_ is 'xcomp':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        excomp_pairs.append(token.text+' '+k.text+' '+j.text)
        pairs = list(set(amod_pairs+advmod_pairs+neg_pairs+xcomp_pairs))
        epairs = list(set(eamod_pairs+eadvmod_pairs+eneg_pairs+excomp_pairs))
        for i in range(len(pairs)):
            if len(compound_pairs)!=0:
                for comp in compound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(pairs[i]))
                    if mtch is not None:
                        pairs[i] = pairs[i].replace(mtch.group(),comp[0])
        for i in range(len(epairs)):
            if len(ecompound_pairs)!=0:
                for comp in ecompound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(epairs[i]))
                    if mtch is not None:
                        epairs[i] = epairs[i].replace(mtch.group(),comp[0])
            
    aspect_terms.append(pairs)
    comp_terms.append(compound_pairs)
    easpect_terms.append(epairs)
    ecomp_terms.append(ecompound_pairs)
    enemy.append(enemlist)
toy_rev['compound_nouns'] = comp_terms
toy_rev['aspect_keywords'] = aspect_terms
toy_rev['competition'] = enemy
toy_rev['competition_comp_nouns'] = ecomp_terms
toy_rev['competition_aspects'] = easpect_terms
toy_rev.head()

100%|███████████████████████████████████████████████████████████████████████████| 78637/78637 [09:21<00:00, 139.99it/s]


,index,Tweets,compound_nouns,aspect_keywords,competition,competition_comp_nouns,competition_aspects
0,0,RT @sanket: It may sound somewhat seditious bu...,"[(RT sanket, sanket)]",[],[],[],[]
1,1,#Mirzapur2 marathon happening in house https:/...,"[(Mirzapur2 marathon, marathon)]",[],[],[],[]
2,2,This match was as disappointing as Mirzapur2. 🤯,[],[],[],[],[]
3,3,#Dream11IPL #MIvRR #Mirzapur2,[],[],[],[],[]
4,4,When you miss your sales targets! #Mirzapur2 h...,"[(sales targets, targets)]",[When miss],[],[],[]


In [7]:
toy_rev[toy_rev['aspect_keywords']!='[]']

,index,Tweets,compound_nouns,aspect_keywords,competition,competition_comp_nouns,competition_aspects
0,0,RT @sanket: It may sound somewhat seditious bu...,"[(RT sanket, sanket)]",[],[],[],[]
1,1,#Mirzapur2 marathon happening in house https:/...,"[(Mirzapur2 marathon, marathon)]",[],[],[],[]
2,2,This match was as disappointing as Mirzapur2. 🤯,[],[],[],[],[]
3,3,#Dream11IPL #MIvRR #Mirzapur2,[],[],[],[],[]
4,4,When you miss your sales targets! #Mirzapur2 h...,"[(sales targets, targets)]",[When miss],[],[],[]
...,...,...,...,...,...,...,...
78632,97956,Le barber --,[],[],[],[],[]
78633,97957,Ft. #Mirzapur2 https://t.co/as5TWpiJAU,[],[],[],[],[]
78634,97958,RT @SanDhirParShFC: Queen Slaying In Blue 👗 @H...,[],[],[],[],[]
78635,97960,#HarshitaGaur #mirzapurseason2 #Mirzapur2 http...,[],[],[],[],[]


In [8]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [9]:
import operator
sentiment = []
for i in range(len(toy_rev)):
    score_dict={'Positive':0,'Negative':0,'Neutral':0}
    if len(toy_rev['aspect_keywords'][i])!=0: 
        for aspects in toy_rev['aspect_keywords'][i]:
            sent = analyser.polarity_scores(aspects)
            score_dict['Negative'] += sent['neg']
            score_dict['Positive'] += sent['pos']
            score_dict['Neutral'] += sent['neu']
        sentiment.append(max(score_dict.items(), key=operator.itemgetter(1))[0])
    else:
        sentiment.append('NaN')
toy_rev['sentiment'] = sentiment
toy_rev.head()

,index,Tweets,compound_nouns,aspect_keywords,competition,competition_comp_nouns,competition_aspects,sentiment
0,0,RT @sanket: It may sound somewhat seditious bu...,"[(RT sanket, sanket)]",[],[],[],[],NaN
1,1,#Mirzapur2 marathon happening in house https:/...,"[(Mirzapur2 marathon, marathon)]",[],[],[],[],NaN
2,2,This match was as disappointing as Mirzapur2. 🤯,[],[],[],[],[],NaN
3,3,#Dream11IPL #MIvRR #Mirzapur2,[],[],[],[],[],NaN
4,4,When you miss your sales targets! #Mirzapur2 h...,"[(sales targets, targets)]",[When miss],[],[],[],Negative


In [10]:
toy_rev['sentiment'].value_counts()

NaN         66195
Neutral      9331
Positive     1856
Negative     1255
Name: sentiment, dtype: int64

In [11]:
toy_rev['sentiment'].value_counts()

NaN         66195
Neutral      9331
Positive     1856
Negative     1255
Name: sentiment, dtype: int64

In [12]:
int_sent = []
for sent in toy_rev['sentiment']:
    if sent is 'NaN':
        int_sent.append('NaN')
    elif sent is 'pos':
        int_sent.append('1')
    else:
        int_sent.append('0')
toy_rev['int_sent'] = int_sent
toy_rev.head()

,index,Tweets,compound_nouns,aspect_keywords,competition,competition_comp_nouns,competition_aspects,sentiment,int_sent
0,0,RT @sanket: It may sound somewhat seditious bu...,"[(RT sanket, sanket)]",[],[],[],[],NaN,NaN
1,1,#Mirzapur2 marathon happening in house https:/...,"[(Mirzapur2 marathon, marathon)]",[],[],[],[],NaN,NaN
2,2,This match was as disappointing as Mirzapur2. 🤯,[],[],[],[],[],NaN,NaN
3,3,#Dream11IPL #MIvRR #Mirzapur2,[],[],[],[],[],NaN,NaN
4,4,When you miss your sales targets! #Mirzapur2 h...,"[(sales targets, targets)]",[When miss],[],[],[],Negative,0


In [14]:
toy_rev.to_csv('ksnsreupdated.csv')

toy_rev.to_csv('ksns_updated.csv')

In [ ]:
toy_rev[toy_rev['int_sent']=='0']['Tweets'].shape

In [ ]:
toy_rev[toy_rev['int_sent']=='1']['Tweets'].shape

In [ ]:
toy_rev[toy_rev.Tweets.isnull()].shape

### Here we have arbitarily taken ratings greater than 3 as positive and everything else as negative

In [127]:
import math
pos = []
for i in range(len(toy_rev)):
    if not math.isnan(toy_rev['Rating'][i]):
        if int(toy_rev['Rating'][i])>3:
            pos.append('1')
        else:
            pos.append('0')
    else:
        pos.append('0')
toy_rev['Positive Review'] = pos
toy_rev.head()

KeyError: 'Rating'

In [21]:
d = {'sent':toy_rev['Positive Review'],'sent_pred':toy_rev['int_sent']}
metric_df = pd.DataFrame(data=d)
metric_df.head()

KeyError: 'Positive Review'

In [ ]:
len(metric_df.sent)

## Removing NaN values in the sentiment predictions

In [ ]:
metric_df = metric_df[metric_df.sent_pred != 'NaN']
len(metric_df.sent)

In [ ]:
from sklearn.metrics import accuracy_score,auc,f1_score,recall_score,precision_score
print('accuracy')
print(accuracy_score(metric_df.sent, metric_df.sent_pred))
print('f1 score')
print(f1_score(metric_df.sent, metric_df.sent_pred,pos_label='1'))
print('recall')
print(recall_score(metric_df.sent, metric_df.sent_pred,pos_label='1'))
print('precision')
print(precision_score(metric_df.sent, metric_df.sent_pred,pos_label='1'))

## Possible improvements that can be made
*  Tricky situation of removing stopwords to reduce unwanted extractions of non-aspects but this can also affect spaCy's dependency parsing. Same goes with noun chunk merging as well. If someone can think of a better way to remove stopwords and still retain spaCy's dependency goodness it can greatly improve the accuracy

* This is not a ML task per se since we do more of parsing than ML. Although Bi-Directional LSTM have been very good at ABSA tasks in the past, unlike semeval tasks we do not have a fixed topic for our aspects to fall into. If someone can use the parsing aspect of the code to implement BLSTM in this case, that would be great

* better alternatives to vaderSentiment if available (unsupervised/ semi-supervised methods might be better here I think)

* The very definition of aspects can be a bit vague at times hence we do not have a valid metric to measure the aspect extraction's accuracy


## P.S This is my first Kaggle Kernel and I am fairly new to python programming as well, hence my non usage of list comprehensions and functions might be evident. I highly encourage everyone to fork my code and add your own twists to increase the accuracy of both aspect extractions and sentiment analysis.